In [10]:
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
import math
import time
from datetime import datetime , timedelta

In [11]:
def connect():
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get("https://www.linkedin.com/checkpoint/lg/sign-in-another-account")
    time.sleep(5)
    username = driver.find_element(By.ID, "username")
    username.send_keys("gaithkrifa123@gmail.com")
    pword = driver.find_element(By.ID, "password")
    pword.send_keys("hedimarzouk1230")
    driver.find_element(By.XPATH, "//button[@type='submit']").click()
    time.sleep(2)
    return driver

In [12]:
def nb_of_page(driver,url):
    
    driver.get(url)

    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # 1. Get number of jobs found and number of pages:
    try:
        div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
        list = div_number_of_jobs.find('span').get_text().strip().split()
        if (len(list)>2):
            number_of_jobs = int(str(list[0])+str(list[1]))
        else:
            number_of_jobs = int(list[0])
    except:
        number_of_jobs = 0
        
    number_of_pages=math.ceil(number_of_jobs/25)
    print("number_of_jobs:",number_of_jobs)
    print("number_of_pages:",number_of_pages)

    return  number_of_pages,soup

In [13]:
def nb_of_page(driver,url):
    
    driver.get(url)

    time.sleep(1)

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # 1. Get number of jobs found and number of pages:
    try:
        div_number_of_jobs = soup.find("div",{"class":"jobs-search-results-list__subtitle"})
        list = div_number_of_jobs.find('span').get_text().strip().split()
        if (len(list)>2):
            number_of_jobs = int(str(list[0])+str(list[1]))
        else:
            number_of_jobs = int(list[0])
    except:
        number_of_jobs = 0
        
    number_of_pages=math.ceil(number_of_jobs/25)
    print("number_of_jobs:",number_of_jobs)
    print("number_of_pages:",number_of_pages)

    return  number_of_pages,soup

In [14]:
def find_Job_Ids(soup):

    Job_Ids_on_the_page = []
    try:
        soup = soup.find('ul',{'class':'scaffold-layout__list-container'})
        job_postings = soup.find_all('li', {'class': 'jobs-search-results__list-item'})
        for job_posting in job_postings:
            Job_ID = job_posting.get('data-occludable-job-id')
            Job_Ids_on_the_page.append(Job_ID)
    except:
        pass

    
    return Job_Ids_on_the_page 

In [15]:
def get_data(driver, job_title = 'software',location = 'Tunisia'):
    List_Job_IDs = []

    url = f'https://www.linkedin.com/jobs/search/?keywords={job_title}&location={location}&f_TPR=r604800'
    url = requests.utils.requote_uri(url)

    number_of_pages, soup = nb_of_page(driver,url)


    if number_of_pages>=1:
        
        for page_num in range(0,number_of_pages):
            try:
                print(f"Scraping page: {page_num+1}",end="...\r")

                if(page_num == 0):
                    List_Job_IDs.extend(find_Job_Ids(soup))

                else:
                    url = f'https://www.linkedin.com/jobs/search/?keywords={job_title}&location={location}&start={25 * page_num}&f_TPR=r604800'
                    url = requests.utils.requote_uri(url)
                    driver.get(url)
                    time.sleep(5)           
                    soup = BeautifulSoup(driver.page_source, 'html.parser')
                    List_Job_IDs.extend(find_Job_Ids(soup)) 
                    
                    if soup.find('ul',{'class':'scaffold-layout__list-container'}) is None:
                        print('End page')
                        break
                
                print("done!")

            except:
                print('Error links')
    
    job={}
    list_jobs=[]

    for j in range(0,len(List_Job_IDs)):
        try:
            if (j%50==0):
                print(f"{j} Jobs read",end="\r")

            job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{List_Job_IDs[j]}"
            resp = requests.get(job_url)    
            soup=BeautifulSoup(resp.text,'html')

            try:
                job['job_link'] = soup.find('a',{'class':'topcard__link'}).get('href')
            except:
                job['job_link'] = None
            
            try:
                job["job_name"]=soup.find("h2",{"class":"top-card-layout__title"}).text.strip()
            except:
                job["job_name"]=None

            
            try: 
                job["job_text"] = soup.find('div',{'class':'description__text description__text--rich'}).text.strip().split("\n")[0]
            except:
                job["job_text"] = None
            
            try:
                job["job_company"]=soup.find("div",{"class":"top-card-layout__card"}).find("a").find("img").get('alt')
            except:
                job["job_company"]=None

            try:
                job["job_location"]=soup.find("span",{"class":"topcard__flavor topcard__flavor--bullet"}).text.strip()
            except:
                job["job_location"]=None

            try:
                job["job_type"]=soup.find("ul",{"class":"description__job-criteria-list"}).find("li").text.replace("Seniority level","").strip()
            except:
                job["job_type"]=None

            try:
                date = soup.find("span",{"class":"posted-time-ago__text"}).text.strip()

                now = datetime.now()
                if 'hour' in date or 'hours' in date or 'minutes' in date or 'minutes' in date :
                    formatted_date = now.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '1 days' in date.replace('\xa0', ''):
                    date = now - timedelta(days=1)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '2 days' in date.replace('\xa0', ''):
                    date = now - timedelta(days=2)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '3 days' in date.replace('\xa0', ''):
                    date = now - timedelta(days=3)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '4 days' in date.replace('\xa0', ''):
                    date = now - timedelta(days=4)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '5 days' in date.replace('\xa0', ''):
                    date = now - timedelta(days=5)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif '6 days' in date.replace('\xa0', ''):
                    date = now - timedelta(days=6)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
                elif 'week' in date.replace('\xa0', ''):
                    date = now - timedelta(days=7)
                    formatted_date = date.strftime("%Y-%m-%d")
                    job['job_date'] = formatted_date
            except:
                job["job_date"]=None

        
            if(len(job)):
                list_jobs.append(job)    
        except:
            print(f"{j+1} ... error read Job ID: {List_Job_IDs[j]}")
            
        
        job={}

    jobs_df = pd.DataFrame(list_jobs)
    return jobs_df


In [16]:
driver=connect()

In [17]:
jobs_df = get_data(driver)

number_of_jobs: 43
number_of_pages: 2
done!ing page: 1...
done!ing page: 2...


In [18]:
jobs_df

,job_link,job_name,job_text,job_company,job_location,job_type,job_date
0,https://tn.linkedin.com/jobs/view/comptable-at...,Comptable,Nous recherchons un comptable compétent et mot...,Sunrise Palace Hôtel,"Délégation Djerba Houmet Souk, Médenine, Tunisia",Mid-Senior level,2024-07-11
1,https://tn.linkedin.com/jobs/view/marketing-et...,Marketing et Photographie pour Chambres d'Hôtel,Nous recherchons un professionnel créatif et p...,Sunrise Palace Hôtel,"Délégation Djerba Houmet Souk, Médenine, Tunisia",Entry level,2024-07-11
2,https://tn.linkedin.com/jobs/view/ing%C3%A9nie...,Ingénieur / Technicien Supérieur Fluide,Nous sommes à la recherche d'un Ingénieur / Te...,Sunrise Palace Hôtel,"Délégation Djerba Houmet Souk, Médenine, Tunisia",Entry level,2024-07-11
3,https://tn.linkedin.com/jobs/view/financier-at...,Financier,Nous recherchons un Financier talentueux et ex...,Sunrise Palace Hôtel,"Délégation Djerba Houmet Souk, Médenine, Tunisia",Mid-Senior level,2024-07-11
4,None,None,None,None,None,None,None
5,https://tn.linkedin.com/jobs/view/administrate...,Administrateur de base de données/Administratr...,Les postes sont basés à Tunis (Lac 1) LOG Syst...,LOG SYSTEM North Africa,"Tunis, Tunisia",Associate,2024-07-05
6,https://tn.linkedin.com/jobs/view/senior-qa-le...,Senior QA Lead (Automation),"iObeya, c’est une aventure qui grandit depuis ...",iObeya,"Tunis, Tunis, Tunisia",Mid-Senior level,2024-07-09
7,https://tn.linkedin.com/jobs/view/d%C3%A9velop...,Développeur FullStack Java/Angular | Summer Jo...,"Description de l'entrepriseChez Devoteam, nous...",Devoteam Tunisie,"Tunis, Tunis, Tunisia",Mid-Senior level,2024-07-06
8,https://tn.linkedin.com/jobs/view/senior-net-s...,Senior .Net SQL Developer,About UsWHY Linedata?Linedata is an award-winn...,Linedata,"Tunis, Tunis, Tunisia",Entry level,2024-07-06
9,https://tn.linkedin.com/jobs/view/ing%C3%A9nie...,Ingénieur QA Fonctionnel - Tunis,CONTEXTEDans le cadre du renforcement de ses é...,NeoXam,"Tunis, Tunisia",Associate,2024-07-09


In [20]:
def get_datas(driver,keywords,location):
    df_job = pd.DataFrame()    
    for i in keywords:
        for j in location:
            df= get_data(driver,i,j)
            df_job=pd.concat([df_job,df],axis=0)
    return df_job

In [21]:
def get_data_Linked(driver, Keywords, Locations):
    data = pd.DataFrame()
    for keyword in Keywords:
        for location in Locations:
            df = get_data(driver,keyword,location)
            data = pd.concat([data,df],axis=0)
    return data

In [22]:
driver = connect()

In [23]:

Keywords = ['data analyst','data scientist','data engineer','web developer','mobile developer']
Locations = ['Tunisia','France']

In [24]:
df_linked = get_data_Linked(driver, Keywords, Locations)

number_of_jobs: 4
number_of_pages: 1
done!ing page: 1...
number_of_jobs: 443
number_of_pages: 18
done!ing page: 1...
done!ing page: 2...
done!ing page: 3...
done!ing page: 4...
done!ing page: 5...
done!ing page: 6...
done!ing page: 7...
done!ing page: 8...
done!ing page: 9...
done!ing page: 10...
done!ing page: 11...
done!ing page: 12...
done!ing page: 13...
done!ing page: 14...
done!ing page: 15...
done!ing page: 16...
done!ing page: 17...
done!ing page: 18...
number_of_jobs: 6
number_of_pages: 1
done!ing page: 1...
number_of_jobs: 573
number_of_pages: 23
done!ing page: 1...
done!ing page: 2...
done!ing page: 3...
done!ing page: 4...
done!ing page: 5...
done!ing page: 6...
done!ing page: 7...
done!ing page: 8...
done!ing page: 9...
done!ing page: 10...
done!ing page: 11...
done!ing page: 12...
done!ing page: 13...
done!ing page: 14...
done!ing page: 15...
done!ing page: 16...
done!ing page: 17...
done!ing page: 18...
done!ing page: 19...
done!ing page: 20...
done!ing page: 21...
done!

In [25]:
df_linked

,job_link,job_name,job_text,job_company,job_location,job_type,job_date
0,https://tn.linkedin.com/jobs/view/football-sta...,Football Statistician (Data Collection),"WHO ARE WE?Genius Sports is the official data,...",Genius Sports,"Tataouine, Tunisia",Not Applicable,2024-07-09
1,https://www.linkedin.com/jobs/view/principal-b...,Principal Biostatistician,We are currently supporting a global leader in...,Hyper Recruitment Solutions,EMEA,Mid-Senior level,2024-07-09
2,https://www.linkedin.com/jobs/view/lead-biosta...,Lead Biostatistician,We are currently supporting a global pharmaceu...,Hyper Recruitment Solutions,EMEA,Mid-Senior level,2024-07-09
3,https://www.linkedin.com/jobs/view/senior-bios...,Senior Biostatistician,We are currently supporting a global leader in...,Hyper Recruitment Solutions,EMEA,Mid-Senior level,2024-07-09
0,https://fr.linkedin.com/jobs/view/internship-d...,Internship - Data Analyst,Skill Farm Members are invited to apply for th...,Skill Farm,"Paris, Île-de-France, France",Internship,NaN
...,...,...,...,...,...,...,...
383,https://fr.linkedin.com/jobs/view/ing%C3%A9nie...,Ingénieur DevOps 45 - 60 K€ H/F,MissionVous avez une expérience d'ingénieur DE...,DigitValue,"Maritime Alps, Provence-Alpes-Côte d'Azur, France",Entry level,2024-07-11
384,https://fr.linkedin.com/jobs/view/ing%C3%A9nie...,Ingénieur IVVQ Cloud H/F,Ingénieur IVVQ Cloud Domaine Bancaire H/FLieu ...,LesJeudis,"Gémenos, Provence-Alpes-Côte d'Azur, France",Entry level,2024-07-11
385,https://fr.linkedin.com/jobs/view/ing%C3%A9nie...,Ingénieur DevOps junior,"En qualité d'ingénieur DevOps, vous développez...",Free-Work,"Le Pecq, Île-de-France, France",Entry level,2024-07-08
386,None,None,None,None,None,None,None


In [26]:
df_linked.shape

(2177, 7)

In [27]:
df_linked.head()

,job_link,job_name,job_text,job_company,job_location,job_type,job_date
0,https://tn.linkedin.com/jobs/view/football-sta...,Football Statistician (Data Collection),"WHO ARE WE?Genius Sports is the official data,...",Genius Sports,"Tataouine, Tunisia",Not Applicable,2024-07-09
1,https://www.linkedin.com/jobs/view/principal-b...,Principal Biostatistician,We are currently supporting a global leader in...,Hyper Recruitment Solutions,EMEA,Mid-Senior level,2024-07-09
2,https://www.linkedin.com/jobs/view/lead-biosta...,Lead Biostatistician,We are currently supporting a global pharmaceu...,Hyper Recruitment Solutions,EMEA,Mid-Senior level,2024-07-09
3,https://www.linkedin.com/jobs/view/senior-bios...,Senior Biostatistician,We are currently supporting a global leader in...,Hyper Recruitment Solutions,EMEA,Mid-Senior level,2024-07-09
0,https://fr.linkedin.com/jobs/view/internship-d...,Internship - Data Analyst,Skill Farm Members are invited to apply for th...,Skill Farm,"Paris, Île-de-France, France",Internship,NaN


In [129]:
df_linked_cleaned = df_linked.dropna()
df_linked_cleaned.shape


(524, 8)

In [28]:
csv_filename = "linkedin.csv"
df_linked.to_csv(csv_filename, index=False)
print(f"Le fichier CSV a été créé avec succès sous le nom {csv_filename}.")


Le fichier CSV a été créé avec succès sous le nom linkedin.csv.


In [130]:
travaux_en_france = df_linked[df_linked['location'].str.contains('France', na=False)]


In [131]:
travaux_en_france.shape

(1372, 8)

In [132]:
len(travaux_en_france)

1372

In [133]:
travaux_en_tunisie= df_linked[df_linked['location'].str.contains('Tunisia', na=False)]
travaux_en_tunisie.shape


(113, 8)